# Preparing and Uploading Data to AWS

These instructions are indended as a starting point. Please add to them if you find steps that are missing etc.

**Date** June 2018

**Author** Claire Krause

## Get an AWS account

Before you can do anything, you need to get a GA AWS account. This will give you permissions to move data onto the DEA AWS S3 bucket. Speak to DEA's cloud engineer to organise an account. They can also point you to the correct place to load data, and will make sure the data you intend to upload can be accommodated.

## Turn the required data into cog tif files

The preferred format for datasets on S3 is cog format, aka 'cloud optimised geoTIFF'. This file format is smaller than netCDF, enables pyramid loading, and is supported by `rasterio`.

### Translating data into cog format

[Harshu](https://github.com/harshurampur) has written some helper functions for translating data into cog format. This code can be found in the [Geotiff-conversion repo](https://github.com/harshurampur/Geotiff-conversion). To use this code, clone the repo into a suitable location (I used my folder on `/g/data/...`). For translation, there are two functions to choose from, depending on the current format of the data you want to upload: 
* `geotiff-cog.py` - for translating GeoTIFF to cog geoTIFFs
* `netcdf-cog.py` - for translating netCDF to cog geoTIFFs

To run the translation scripts, you will need to specify the path of the input and output files. You can then run the scripts from the command line. (You will need to have the DEA library loaded as this script requires some libraries to run):

`python geotiff-cog.py -p /path/to/directory/of/files/for/translation -o /path/to/output/directory`

The script will create a directory in the output location that mirrors the directory name containing the input files. This script can handle data with multiple bands and time steps. A new cog `.tif` file is created for each timestep and band, so this code can take a while to run...

## Copy the data to aws

### Add your aws credentials to raijin:

In order to run aws commands from within raijin, you will need to have your credentials and configuration files set up in your raijin home directory.

Your credientials can be found by opening your AWS account, and selecting `IAM`. Select `Users` on the left-hand panel, then search for your u-number. Once you click on your u-number, you will see a box to `Add new security keys`. Select this, and then copy the access key and secret access key into the credentials file explained below.

`cd ~`

`mkdir .aws`

`nano .aws/credentials`
```
[default]
aws_access_key_id=
aws_secret_access_key=
```
*(Ctrl-O to save, Ctrl-X to exit)*

`nano .aws/config`
```
[default]
region=ap-southeast-2
```
*(Ctrl-O to save, Ctrl-X to exit)*

### Run the PBS script to sync your data with aws

The code for doing this can be found in the same [Geotiff-conversion repository](https://github.com/harshurampur/Geotiff-conversion/tree/master/compute-sync). Modify the `compute_sync.sh` code with your own username, and update last line the command to point to the two directories you want to sink;
- the directory on NCI with the prepared data
- the folder on S3 where you want the data to go

### Hit go!

Once you have set up your aws credentials, and modified `compute_sync.sh`, submit this script to PBS (either from within the `Geotiff-conversion/compute-sync` folder, or from anywhere on raijin with the full or relative path to the `compute_sync.sh` file included):

`qsub compute_sync.sh`